In [1]:
import cv2
import dlib
import keyboard
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier

In [2]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


In [4]:
cap = cv2.VideoCapture(0)

mood = input("Enter your mood : ")

frames = []
outputs = []

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        # print(landmarks.parts())
        nose = landmarks.parts()[27]
        # print(nose.x, nose.y)

        expression = np.array([[point.x, point.y] for point in landmarks.parts()[0:47]])
        for point in landmarks.parts():
            cv2.circle(frame, (point.x,point.y),2,(255,0,0),3)


    # print(faces)

    if ret:
        cv2.imshow("My Screen", frame)

    key = cv2.waitKey(1)

    if key == ord("q"):
        break
    elif key == ord("c"):
        frames.append(expression.flatten())
        outputs.append([mood])


X = np.array(frames)
y = np.array(outputs)

data = np.hstack([y, X])

f_name = "face_data.npy"

if os.path.exists(f_name):
    old = np.load(f_name)
    data = np.vstack([old, data])

np.save(f_name, data)

cap.release()
cv2.destroyAllWindows()

Enter your mood :  ll


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 95 and the array at index 1 has size 0

In [4]:
cap = cv2.VideoCapture(0)

data=np.load("face_data.npy")
X=data[:,1:].astype(int)
y=data[:,0]
model=KNeighborsClassifier()
model.fit(X,y)

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)
        # print(landmarks.parts())
     
        # print(nose.x, nose.y)
        lipUp=landmarks.parts()[62]
        lipDown=landmarks.parts()[66]
        expression = np.array([[point.x, point.y] for point in landmarks.parts()[0:47]])
        #for point in landmarks.parts():
            #cv2.circle(frame, (point.x,point.y),2,(255,0,0),3)
        if model.predict([expression.flatten()])=="left":
            keyboard.press("left")
            #print("left")
        elif model.predict([expression.flatten()])=="right":
            keyboard.press("right")
            #print("right")
        elif model.predict([expression.flatten()])=="center":
            keyboard.release("right")
            keyboard.release("left")
        if(lipDown.y-lipUp.y>5):
            keyboard.press("down")
            #print("")
        
        else:
            keyboard.press("up")
            #print("")
        #print(expression.flatten())


    # print(faces)

    if ret:
        cv2.imshow("My Screen", frame)

    key = cv2.waitKey(1)

    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()